In [271]:
from typing import List

import nltk
from pandas import DataFrame
import pandas as pd
import spacy
import re
import openpyxl
! python -m spacy download es_core_news_sm
# ! python -m spacy download es_core_news_lg

  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.5.0/es_core_news_sm-3.5.0-py3-none-any.whl (12.9 MB)

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [272]:
def print_corpus(corpus: List[str]):
    for i in range(len(corpus)):
        text = corpus[i]
        print()
        print(f'\033[1;32;40m {"-" * 10} Text: {str(i + 1)} {"-" * 10} \033[0m')
        print(text)

In [273]:
def colorize(text: str) -> str:
    text = text.replace('$phone$', "\033[1;33;40m$phone$\033[0m")
    text = text.replace('$person$', "\033[1;33;40m$person$\033[0m")
    text = text.replace('$email$', "\033[1;33;40m$email$\033[0m")
    return text

In [274]:
print("\033[1;34;40m 0. Importar corpus \033[0m")

file: str = "corpus_primer_parcial.xlsx"
df: DataFrame = pd.read_excel(file)
the_corpus: List[str] = df.iloc[:, 0].tolist()
the_corpus = [the_corpus[61]]
print_corpus(the_corpus)

 0. Importar corpus 

 ---------- Text: 1 ---------- 
%18``La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. 9911027120
correo: miguel.gonzalez@correo.uady.mx``


In [275]:
# Ejercicio 1
print("\033[1;34;40m 1. Pasar a minúsculas todo el texto \033[0m")


def to_lower(text: str) -> str:
    return text.lower()


my_texts_lower = [to_lower(text) for text in the_corpus]
print_corpus(my_texts_lower)

 1. Pasar a minúsculas todo el texto 

 ---------- Text: 1 ---------- 
%18``la contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. 9911027120
correo: miguel.gonzalez@correo.uady.mx``


In [276]:
# Ejercicio 2
print(
    "\033[1;34;40m 2. Quitar los nombres propios que aparezcan y sustituirlos por una etiqueta llamada #Persona. \033[0m")

print()

print("\033[1;34;40m \t2.1. Detectar nombres \033[0m")

nlp = spacy.load("es_core_news_sm")


def fill_text(text: str) -> str:
    # garantizar espacios en los paréntesis
    text = re.sub(r'(\S)\(', r'\1 (', text)
    text = re.sub(r'\)(\S)', r') \1', text)
    # garantizar espacios en los guiones si es adyacente a una letra
    # text = re.sub(r'-(\w)', r'- \1', text)
    # text = re.sub(r'(\w)-', r'\1 -', text)
    text = re.sub(r'(\S)-', r'\1 -', text)
    text = re.sub(r'-(\S)', r'- \1', text)
    # print(text)
    return text


def get_names(text: str):
    result = set()
    text = fill_text(text)
    text_lines = text.splitlines()
    for line in text_lines:
        doc = nlp(line)
        for ent in doc.ents:
            if 'PER' == ent.label_:
                result.add(ent.text)
    return result


def print_names(names):
    for i in range(len(names)):
        name = names[i]
        print()
        print(f'\033[1;32;40m {"-" * 10} Text: {str(i + 1)} (Names) {"-" * 10} \033[0m')
        print(name)


detected_names = []
for text in the_corpus:
    detected_names.append(get_names(text))
print_names(detected_names)

print()

print("\033[1;34;40m \t2.1. Reemplazar nombres \033[0m")


def replace_names(text: str, names=[]) -> str:
    if 0 == len(names):
        names = get_names(text)
    for name in names:
        text = text.replace(name, '$person$')
    return text


my_texts_no_names = []
for i in range(len(the_corpus)):
    text = the_corpus[i]
    names = detected_names[i]
    text = replace_names(text, names)
    text = text.replace('$person$', "\033[1;33;40m$person$\033[0m")
    my_texts_no_names.append(text)
print_corpus(my_texts_no_names)

 2. Quitar los nombres propios que aparezcan y sustituirlos por una etiqueta llamada #Persona. 

 	2.1. Detectar nombres 

 ---------- Text: 1 (Names) ---------- 
set()

 	2.1. Reemplazar nombres 

 ---------- Text: 1 ---------- 
%18``La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. 9911027120
correo: miguel.gonzalez@correo.uady.mx``


In [277]:
# Ejercicio 3
print("\033[1;34;40m 3. Eliminar códigos que sean basura, ejemplo: %18``.\033[0m")


def remove_garbage(text: str) -> str:
    # garantizar espacios en los paréntesis
    text = re.sub(r"(`)\1+", '', text)
    text = re.sub(r'(%\d{2})', '', text)
    return text


my_texts_no_garbage = [remove_garbage(text) for text in the_corpus]
print_corpus(my_texts_no_garbage)

 3. Eliminar códigos que sean basura, ejemplo: %18``.

 ---------- Text: 1 ---------- 
La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. 9911027120
correo: miguel.gonzalez@correo.uady.mx


In [278]:
# Ejercicio 4
print("\033[1;34;40m 4. Eliminar números telefónicos y sustituirlos por #Teléfono.\033[0m")


def remove_phone_numbers(text: str) -> str:
    return re.sub(r'([\(\s])\d{10}([\)\s])', r'\1$phone$\2', text)


my_texts_no_phone_numbers = []
for text in the_corpus:
    text = remove_phone_numbers(text)
    text = text.replace('$phone$', "\033[1;33;40m$phone$\033[0m")
    my_texts_no_phone_numbers.append(text)
print_corpus(my_texts_no_phone_numbers)

 4. Eliminar números telefónicos y sustituirlos por #Teléfono.

 ---------- Text: 1 ---------- 
%18``La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. $phone$
correo: miguel.gonzalez@correo.uady.mx``


In [279]:
# Ejercicio 5
print("\033[1;34;40m 5. Eliminar correos electrónicos y sustituirlos por #Email.\033[0m")


def remove_emails(text: str) -> str:
    return re.sub(r'([\w\.-]+@[\w\.-]+)', r'$email$', text)


my_texts_no_emails = []
for text in the_corpus:
    text = remove_emails(text)
    text = text.replace('$email$', "\033[1;33;40m$email$\033[0m")
    my_texts_no_emails.append(text)
print_corpus(my_texts_no_emails)

 5. Eliminar correos electrónicos y sustituirlos por #Email.

 ---------- Text: 1 ---------- 
%18``La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. 9911027120
correo: $email$``


In [280]:
# Análisis completo
print("\033[1;34;40m Análisis completo \033[0m")

for i in range(len(the_corpus)):
    text = the_corpus[i]
    print()
    print(f'\033[1;32;40m {"-" * 10} Text: {str(i + 1)} {"-" * 10} \033[0m')
    print()
    print("\033[1;35;40m Original: \033[0m")
    print(text)
    print()
    print("\033[1;35;40m Garbage Removed: \033[0m")
    text = remove_garbage(text)
    print(text)
    print()
    print("\033[1;35;40m Phones Removed: \033[0m")
    text = remove_phone_numbers(text)
    print(colorize(text))
    print()
    print("\033[1;35;40m Emails Removed: \033[0m")
    text = remove_emails(text)
    print(colorize(text))
    print()
    print("\033[1;35;40m Names Removed: \033[0m")
    text = replace_names(text)
    print(colorize(text))
    print()
    print("\033[1;35;40m Lower: \033[0m")
    text = to_lower(text)
    print(colorize(text))
    print()

 Análisis completo 

 ---------- Text: 1 ---------- 

 Original: 
%18``La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. 9911027120
correo: miguel.gonzalez@correo.uady.mx``

 Garbage Removed: 
La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. 9911027120
correo: miguel.gonzalez@correo.uady.mx

 Phones Removed: 
La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. $phone$
correo: miguel.gonzalez@correo.uady.mx

 Emails Removed: 
La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. $phone$
correo: $email$

 Names Removed: 
La contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ingresar  

cel. $phone$
correo: $email$

 Lower: 
la contraseña de mi correo no es aceptada ya es la segunda ocasión que se bloquea y no puedo ing